In [ ]:
# from libs import stable_diffusion_service
from libs.stable_diffusion_service import GenerateImage, StableDiffusion
from PIL import Image
from IPython.display import display
from tqdm import tqdm
from tqdm.asyncio import tqdm as async_tqdm
import asyncio
import time
import os
import csv
import pandas as pd

# STABLE_DIFFUSION_BASE_URL = "http://100.119.72.3:7860" # 腾讯云IDE
STABLE_DIFFUSION_BASE_URL = "http://192.168.10.101:7860"  # 笔记本
# STABLE_DIFFUSION_BASE_URL = "http://localhost:6006" # AutoDL

SD = StableDiffusion(STABLE_DIFFUSION_BASE_URL, concurrency_limit=10)

In [12]:
# 读取 csv 文件并构建 prompt 列表
sd_style_list: pd.DataFrame = pd.read_csv("csv/300_NAI_Styles_Table.csv")
common_prompts: pd.DataFrame = pd.read_csv("csv/common_prompts.csv")
prompt_list: list[str] = []

for _, sd_style in sd_style_list.iterrows():
    for _, common_prompt in common_prompts.iterrows():
        prompt = (
            f"{common_prompt['Gender tags']}"
            f"{common_prompt['Character(s) tags']}"
            f"{common_prompt['Series tags']}"
            f"{common_prompt['Rating tags']}"
            f"{sd_style['Artists']}"
            f"{common_prompt['General tags']}"
            f"{common_prompt['Qulity tags']}"
        )
        prompt_list.append(prompt)

print(f"Prompt list length: {len(prompt_list)}")
for idx, prompt in enumerate(prompt_list[:6]):
    print(f"Prompt {idx}: {prompt}")

Prompt list length: 868
Prompt 0: 1girl,amiya \(arknights\),arknights,safe,(mafuyu \(chibi21\)),((rurudo,gusha s,hiro \(dismaless\),wlop,yoneyama mai,mikami mika)),[[murata range]],solo,cowboy shot,smile,looking at viewer,masterpiece,high score,great score,absurdres,year 2025,
Prompt 1: 2girls,texas \(arknights\),lappland \(arknights\),arknights,safe,(mafuyu \(chibi21\)),((rurudo,gusha s,hiro \(dismaless\),wlop,yoneyama mai,mikami mika)),[[murata range]],shopping mall,indoor,cowboy shot,holding hands,front view,masterpiece,high score,great score,absurdres,year 2025,
Prompt 2: 1girl,amiya \(arknights\),arknights,safe,naga u,[tyakomes],henreader,baku-psolo,cowboy shot,smile,looking at viewer,masterpiece,high score,great score,absurdres,year 2025,
Prompt 3: 2girls,texas \(arknights\),lappland \(arknights\),arknights,safe,naga u,[tyakomes],henreader,baku-pshopping mall,indoor,cowboy shot,holding hands,front view,masterpiece,high score,great score,absurdres,year 2025,
Prompt 4: 1girl,amiya 